In [ ]:
import pandas as pd
import google.generativeai as genai
import time
import os
from tqdm import tqdm
import json
# Cấu hình API Key
genai.configure(api_key="")  # <-- Nhớ điền API key ở đây

# Khởi tạo model
model = genai.GenerativeModel(model_name='models/gemini-1.5-flash-latest')

# File input và output
input_file = "../document_dataset/document_chunk_dataset.xlsx"
output_file = "SVYKHOA_dataset.xlsx"

# Đọc dữ liệu đầu vào
try:
    df_input = pd.read_excel(input_file)
    df_input = df_input[['Text']].dropna().reset_index(drop=True)
    print(f"Đã đọc file Excel với {len(df_input)} dòng văn bản.")
except Exception as e:
    print(f"Lỗi khi đọc file Excel: {str(e)}")
    exit()

# Tạo DataFrame kết quả
result_columns = [
    "icd_10", "question", "symptom", "diagnosis",
    "document/title", "document/tool", "document/description",
    "cme/title", "cme/tool", "cme/description"
]
if os.path.exists(output_file):
    df_result = pd.read_excel(output_file)
else:
    df_result = pd.DataFrame(columns=result_columns)

# Prompt Template mới
prompt_template = """
Bạn là một chuyên gia y tế chuyên về y khoa. Nhiệm vụ của bạn là tạo dữ liệu huấn luyện dành cho một **chatbot y khoa chuyên về y khoa** nhằm giúp chatbot phản hồi người dùng một cách chính xác, khoa học và đáng tin cậy theo chuẩn ICD-10 của Bộ Y tế Việt Nam.

Dựa vào đoạn văn sau:

"{dental_text}"

Hãy sinh ra một **DANH SÁCH GỒM 3 ĐỐI TƯỢNG JSON** chứa đầy đủ TẤT CẢ các trường dưới đây. Tuyệt đối không được bỏ trống bất kỳ trường nào. Nếu đoạn văn không nêu rõ thông tin, bạn được phép **bổ sung hợp lý dựa trên kiến thức y khoa chính thống**, nhưng KHÔNG được bịa đặt hoặc sử dụng thông tin sai sự thật.

**Mục tiêu**: Dữ liệu được tạo ra sẽ được dùng để huấn luyện một **mô hình chatbot y tế**, vì vậy tất cả thông tin cần:
- Chính xác về mặt y học.
- Bám sát nội dung đoạn văn hoặc kiến thức y khoa nha khoa phổ biến đã được xác thực (ví dụ: WHO, Bộ Y tế, giáo trình nha khoa).
- Có giá trị hướng dẫn, giải thích rõ ràng, có thể dùng để **tư vấn lâm sàng cho người dùng cuối**.

**Yêu cầu đặc biệt**:
- Nếu đoạn văn có đề cập đến các **chỉ số lâm sàng** như độ sâu túi lợi, chỉ số mảng bám, số răng tổn thương, mức độ đau... thì **phải đưa các con số và dữ liệu đó vào `symptom` và `diagnosis`** để tăng độ chính xác cho chatbot.
- `symptom` và `diagnosis` phải viết theo ngôn ngữ chuyên môn, **khoa học, cụ thể, không chung chung**, đủ để một bác sĩ hoặc chatbot sử dụng để diễn giải cho người bệnh.

Các trường cần điền:

- "icd_10": Mã bệnh ICD-10 của Bộ Y tế Việt Nam phù hợp nhất với đoạn văn (ví dụ: "A01.1", "P27.0") nhưng nếu không xác định được thì gán chõ mã bệnh gần đúng nhất.
- "question": Một câu hỏi y khoa phù hợp với **nội dung bài viết và câu trả lời**, tập trung vào chẩn đoán hoặc điều trị y khoa (giống như người dùng hỏi chatbot).
- "symptom": Các triệu chứng hoặc dấu hiệu lâm sàng được nêu trong bài, hoặc suy luận có cơ sở khoa học. **Bao gồm cả các chỉ số định lượng nếu có** như: túi nha chu 6mm, PI=2.2, v.v.
- "diagnosis": Chẩn đoán nha khoa chi tiết, có thể phân loại theo mức độ bệnh hoặc giai đoạn nếu phù hợp. **Phải có cơ sở y khoa vững chắc** và dùng thông số hỗ trợ chẩn đoán nếu có.
- "document/title": Tiêu đề tài liệu y khoa liên quan đến nội dung đoạn văn.
- "document/tool": Luôn là "call_references"
- "document/description": Mô tả ngắn gọn tài liệu tham khảo, nêu rõ mục đích hoặc giá trị ứng dụng.
- "cme/title": Tiêu đề khóa học đào tạo y khoa liên tục (CME) liên quan đến chủ đề trong đoạn văn.
- "cme/tool": Luôn là "call_cme"
- "cme/description": Mô tả ngắn về khóa học CME, giúp chatbot hiểu và phản hồi như một trợ lý học thuật chuyên ngành.

Lưu ý:Chỉ xuất đầu ra dưới dạng đối tượng JSON hợp lệ. KHÔNG thêm bất kỳ mô tả, lời giải thích hoặc đoạn văn nào khác bên ngoài JSON.
"""



# Hàm sinh dữ liệu
def generate_with_retry(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            if response.text:
                return response
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 5
                print(f"Lỗi, thử lại sau {wait_time}s... (Lần {attempt + 1})")
                time.sleep(wait_time)
            else:
                print(f"Lỗi sau {max_retries} lần thử: {str(e)}")
                return None
    return None

# Hàm xử lý và lưu
def process_and_save(response):
    try:
        cleaned = response.text.replace('```json', '').replace('```', '').strip()
        data_list = json.loads(cleaned)

        if not isinstance(data_list, list):
            raise ValueError("Kết quả không phải là danh sách JSON.")

        global df_result
        for data in data_list:
            new_row = {
                "icd_10": data.get("icd_10", ""),
                "question": data.get("question", ""),
                "symptom": data.get("symptom", ""),
                "diagnosis": data.get("diagnosis", ""),
                "document/title": data.get("document/title", ""),
                "document/tool": data.get("document/tool", ""),
                "document/description": data.get("document/description", ""),
                "cme/title": data.get("cme/title", ""),
                "cme/tool": data.get("cme/tool", ""),
                "cme/description": data.get("cme/description", "")
            }
            df_result.loc[len(df_result)] = new_row
        return True
    except Exception as e:
        print(f"\nLỗi xử lý response: {str(e)}")
        print(f"Response gốc: {response.text[:200]}...")
        return False

# Thiết lập số dòng xử lý
start_row = len(df_result)
end_row = len(df_input)
processed_count = 0
start_time = time.time()

# Vòng lặp xử lý
for index in tqdm(range(start_row, end_row)):
    row = df_input.iloc[index]
    retries = 0

    while retries < 3:
        try:
            dental_text = row['Text'].strip()
            prompt = prompt_template.format(dental_text=dental_text)
            response = generate_with_retry(prompt)

            if response and process_and_save(response):
                processed_count += 1
                df_result.to_excel(output_file, index=False)
                time.sleep(2)
                break
            else:
                retries += 1
                print(f"Retry dòng {index} - Lần {retries}")
                time.sleep(3)

        except Exception as e:
            print(f"\nLỗi dòng {index}: {str(e)}")
            retries += 1
            time.sleep(3)

# Lưu cuối
df_result.to_excel(output_file, index=False)
total_time = (time.time() - start_time) / 60
print(f"\nHoàn thành! Đã xử lý {processed_count} dòng.")
print(f"Thời gian chạy: {total_time:.2f} phút")
print(f"File kết quả: {output_file}")


Đã đọc file Excel với 85991 dòng văn bản.


  0%|          | 28/85991 [08:13<384:53:38, 16.12s/it]

Lỗi, thử lại sau 5s... (Lần 1)
Lỗi, thử lại sau 10s... (Lần 2)
Lỗi sau 3 lần thử: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 56
}
]
Retry dòng 28 - Lần 1
Lỗi, thử lại sau 5s... (Lần 1)
Lỗi, thử lại sau 10s... (Lần 2)
Lỗi sau 3 lần thử: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 37
}
]
Retry dòng 28 - Lần 2
Lỗi, thử lại sau 5s... (Lần 1)
Lỗi, thử lại sau 10s... (Lần 2)
Lỗi sau 3 lần thử: 429 You ex

  0%|          | 29/85991 [09:10<674:45:40, 28.26s/it]

Lỗi, thử lại sau 5s... (Lần 1)
Lỗi, thử lại sau 10s... (Lần 2)
Lỗi sau 3 lần thử: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
}
]
Retry dòng 29 - Lần 1
Lỗi, thử lại sau 5s... (Lần 1)
Lỗi, thử lại sau 10s... (Lần 2)
Lỗi sau 3 lần thử: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 41
}
]
Retry dòng 29 - Lần 2
Lỗi, thử lại sau 5s... (Lần 1)
Lỗi, thử lại sau 10s... (Lần 2)
Lỗi sau 3 lần thử: 429 You exceeded your cu

  0%|          | 30/85991 [10:06<878:22:24, 36.79s/it]

Lỗi, thử lại sau 5s... (Lần 1)
Lỗi, thử lại sau 10s... (Lần 2)


  0%|          | 30/85991 [10:22<495:25:34, 20.75s/it]


KeyboardInterrupt: 